In [4]:
import tensorflow as tf
if tf.__version__ != "2.10.0":
    print(f"Current TensorFlow version: {tf.__version__}, switching to 2.10.0")

    # Uninstall current TensorFlow version
    !pip uninstall -y tensorflow

    # Install TensorFlow 2.10
    !pip install tensorflow==2.12

    # After installation, restart runtime
    print("TensorFlow 2.10 installed.")
    print("Please click on the Runtime > Restart session and run all.")
else:
    print("TensorFlow 2.10 is already installed.")

Current TensorFlow version: 2.18.0, switching to 2.10.0
Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.0/586.0 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 27.6

TensorFlow 2.10 installed.
Please click on the Runtime > Restart session and run all.


In [1]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [2]:
l0 = Dense(units=1, input_shape=[1])
model = Sequential([l0])
model.compile(optimizer='sgd', loss='mean_squared_error')

xs = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

model.fit(xs, ys, epochs=500)

print(model.predict(np.array([10.0])))
print("Here is what I learned: {}".format(l0.get_weights()))

Epoch 1/500
1/1 [==============================] - 0s 448ms/step - loss: 10.4966
Epoch 2/500
1/1 [==============================] - 0s 11ms/step - loss: 8.4721
Epoch 3/500
1/1 [==============================] - 0s 8ms/step - loss: 6.8750
Epoch 4/500
1/1 [==============================] - 0s 7ms/step - loss: 5.6141
Epoch 5/500
1/1 [==============================] - 0s 8ms/step - loss: 4.6179
Epoch 6/500
1/1 [==============================] - 0s 7ms/step - loss: 3.8300
Epoch 7/500
1/1 [==============================] - 0s 10ms/step - loss: 3.2061
Epoch 8/500
1/1 [==============================] - 0s 8ms/step - loss: 2.7113
Epoch 9/500
1/1 [==============================] - 0s 10ms/step - loss: 2.3181
Epoch 10/500
1/1 [==============================] - 0s 8ms/step - loss: 2.0050
Epoch 11/500
1/1 [==============================] - 0s 7ms/step - loss: 1.7549
Epoch 12/500
1/1 [==============================] - 0s 7ms/step - loss: 1.5545
Epoch 13/500
1/1 [==============================] - 0s 

In [3]:
export_dir = 'saved_model/1'
tf.saved_model.save(model, export_dir)

In [4]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

In [5]:
import pathlib
tflite_model_file = pathlib.Path('model.tflite')
tflite_model_file.write_bytes(tflite_model)

1080

In [6]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

[{'name': 'serving_default_dense_input:0', 'index': 0, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 3, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [7]:
to_predict = np.array([[10.0]], dtype=np.float32)
print(to_predict)
interpreter.set_tensor(input_details[0]['index'], to_predict)
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])
print(tflite_results)

[[10.]]
[[18.98267]]
